In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import os
import sys
import json
import webbrowser
from json.decoder import JSONDecodeError
import os
import io
from six.moves.urllib.request import urlopen
import numpy as np
import librosa
import librosa.display
import soundfile as sf
import glob
import pickle
import config

import pydub
from urllib.request import urlopen
import requests
from shutil import copyfileobj
from tempfile import NamedTemporaryFile
from urllib.request import urlopen, Request
import matplotlib.pyplot as plt

## Connecting to spotify and obtaining song info

Just going by the spotify playlist criteria was a bad idea- turns out techno playlists dont always have techno songs in them. Same goes for a lot of other genres. Therefore will have to look for the genre of the song specifically as well.

In [8]:
#you will need your own client id and secret
client_id= config.client_id
client_secret =config.client_secret

In [9]:
#this code block is needed to connect to spotify 
username = '1143043561'
auth = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)


try:
    token = auth.get_access_token()
except:
    os.remove(f'.cache-{username}')
    token = auth.get_access_token()
    
#create spotify object
spotify = spotipy.Spotify(auth=token)

In [10]:
def get_songs_from_playlist_search(limit=10,artist_name='techno'):
    '''takes a search term such as techno, finds playlists with techno
        in the name and then returns information about the songs in
        the playlist'''
    playlist_ids=[]
    song_ids = []
    playlists =[]
    song_ids=[]
    preview_urls=[]
    song_names=[]
    
    count=0
    
    results = spotify.search(q=artist_name,limit=limit, type='playlist')
    #find playlists with specific search term
    for playlist_info in results['playlists']['items']:
        playlist_ids.append(playlist_info['id'])
    
    #add all the songs in the playlists to another list
    for playlist in playlist_ids:
        source_playlist=spotify.user_playlist(username,playlist)
        song_set=source_playlist['tracks']
        songs=song_set['items']
        
        while song_set['next']:
            song_set=spotify.next(song_set)
            for song in song_set['items']:
                songs.append(song)
        playlists.append(songs)
    
        
    #loop over the song lists and extract information
    for song_list in playlists:
        for track in song_list:
            
            try:
                if track['track']['id']==None:
                    pass
                else:
                    try:
                        song_ids.append(track['track']['id'])
                        song_names.append(track['track']['name'])
                        preview_urls.append(track['track']['preview_url'])

                    except:
                        print(count)
                        count+=1
            except:
                print(count)
                count+=1
    
    return (song_ids,song_names,preview_urls)

In [11]:
def get_genre(song_ids):
    #get genres using song ids
    genres=[]
    track=spotify.track(song_ids)
    artist_id=track['artists'][0]['id']
    album_id=track['album']['id']
    album_info=spotify.album(album_id)
    #first looks to see if album has genre and if not looks at artist genres
    genres=album_info['genres']
    if not genres:
        artist=spotify.artist(artist_id)
        genres=artist['genres']
    return genres

In [12]:
#make it so there is only one genre
def top_genre(x):
    
    try:
        return x[0].split(' ')[-1]
    except:
        try:
            return x[0]
        except:
            return 'blank'

In [13]:
#filter out the genres that you dont want
def filter_by_genre(songs,genres):
    songs_with_genre=[]
    for val in songs:
        if val[-1] in genres:
            songs_with_genre.append(val)
    return songs_with_genre

In [14]:
search_words=['techno','pop','hip hop','grime','uk garage','rock','calssical']
allowed_genres=[['techno','house','electronic'],['pop','electropop'],
                   ['hop','rap','funk','crunk'], ['grime','rap'],
                   ['garage','dancehall','breakbeat','funky'],
                   ['rock','metal''alternative'],
                   ['classical','performance','baroque','score']]
search_dict={}
for ind, word in enumerate(search_words):
    ids,names,previews=get_songs_from_playlist_search(limit=1,artist_name=word)
    genres=[]
    for song_id in ids:
        genres.append(get_genre(song_id))
    
    clean_genres = []
    for genre_set in genres:
        clean_genres.append(top_genre(genre_set))
    
    #for each song list i am only going to allow certain genres to keep the song list 'pure'
    
                   
    song_zip=list(zip(ids,names,previews,clean_genres))
    song_zip_allowed =filter_by_genre(song_zip,allowed_genres[ind])
    
    search_dict[word]= song_zip_allowed
    
    
    

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [15]:
all_songs_final=list(search_dict.values())

In [16]:
#save so dont have to do this process everytime
with open('new_all_songs1.pkl', 'wb') as f:
    pickle.dump(all_songs_final,f)

In [17]:
with open('new_all_songs1.pkl', 'rb') as f:
     all_songs_final = pickle.load(f)

In [18]:
all_songs_final[0][3][3]

'electronic'

In [19]:
all_songs_final[0][0]

('7v1JbouuzwrFU5waJXR5cu',
 'Ever Gone',
 'https://p.scdn.co/mp3-preview/930acf9e6e46e30df026c95ea38178a738f2ae9f?cid=461b02dc885841c0a798981c336f46f0',
 'techno')

## Creating Spectograms

In [20]:
categories=['techno','pop','hip hop','grime','uk garage','rock','calssical']
full_dict={}

In [21]:
#count to keep track of where we are 


count=0
did_not_convert=0
for ind, category in enumerate(all_songs_final):
    temp_specs=[]
    temp_info=[]
    for ind_song, tup in enumerate(category):
        if tup[2] != None:
            try:
                
                mp3_url = tup[2]  
                wav = io.BytesIO()
                with urlopen(mp3_url) as r:
                    r.seek = lambda *args: None  # allow pydub to call seek(0)
                    pydub.AudioSegment.from_file(r).export(wav, "wav")

                wav.seek(0)
                y, sr = librosa.load(wav)

                # mel-scaled power (energy-squared) spectrogram
                mel_spec = librosa.feature.melspectrogram(y, sr=sr,n_mels=128,hop_length=1024,n_fft=2048)
                # Convert to log scale (dB)
                log_mel_spec = librosa.amplitude_to_db(mel_spec, ref=np.max)
                #make dimensions of the array smaller
                log_mel_spec = np.resize(log_mel_spec,(128,644))
                #prints count every 50 spectograms
                if ind_song%50==0:
                    print(count)
                count+=1
                #store the info
                temp_specs.append(log_mel_spec)
                temp_info.append(tup)
                #the except is just so it doesnt break if one of the urls is wrong/missing
            except:
                    print('did not convert')
                    did_not_convert+=1
    full_dict[categories[ind]]=(temp_specs,temp_info)

0
103
157
227


In [22]:
specs= [tup[0] for tup in full_dict.values()]
info= [tup[1] for tup in full_dict.values()]

In [23]:
#save spectograms because its long to do every time
with open('new_spectros.pkl', 'wb') as f:
    pickle.dump(specs,f)

In [24]:
#save names of songs
with open('new_info.pkl','wb') as f:
    pickle.dump(info,f)